In [1]:
import re
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
import string
from tensorflow import keras
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import numpy as np
import pandas as pd


In [2]:
#Read in the data
import pandas as pd

df1=pd.read_csv("https://raw.githubusercontent.com/statzenthusiast921/political-emails-analysis/main/main/data/clean_emails_df1.csv")
df2=pd.read_csv("https://raw.githubusercontent.com/statzenthusiast921/political-emails-analysis/main/main/data/clean_emails_df2.csv")
df3=pd.read_csv("https://raw.githubusercontent.com/statzenthusiast921/political-emails-analysis/main/main/data/clean_emails_df3.csv")
df4=pd.read_csv("https://raw.githubusercontent.com/statzenthusiast921/political-emails-analysis/main/main/data/clean_emails_df4.csv")

print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)


(19000, 20)
(19000, 20)
(19000, 20)
(17148, 20)


In [3]:
#Stack 4 dataframes together
df = pd.concat([df1, df2, df3, df4], ignore_index=True, axis=0)
#Delete first row and column
df = df[1:]
df = df.iloc[: , 1:]

#Drop NAs from cleaned_body and body
df = df[df['cleaned_body'].notna()]
#Preview dataframe
df.head()


,subject,date,body,party,country,locality,office,time,AM_PM,Hour,Hour_Mil,month,day,year,month_num,cleaned_body,sentiment,compound,comp_score
1,TAKE ACTION for Freedom: #CloseTheCamps,"July 3, 2019","Dear friend, We have all seen the images and r...",NaN,United States,NaN,NaN,11:31 PM,PM,11,23,July,3,2019,7,dear friend seen images read stories migrants ...,"{'neg': 0.207, 'neu': 0.674, 'pos': 0.12, 'com...",-0.9807,0
2,trauma-informed schools,"July 3, 2019","Team,Access to education is fundamental to a c...",Democratic,United States,Ohio,President of the United States,11:32 PM,PM,11,23,July,3,2019,7,team access education fundamental child succes...,"{'neg': 0.143, 'neu': 0.658, 'pos': 0.199, 'co...",0.7269,1
3,Juárez,"July 3, 2019","Beto for America John, The Trump administrati...",Democratic,United States,Texas,President of the United States,11:32 PM,PM,11,23,July,3,2019,7,beto america john trump administration created...,"{'neg': 0.069, 'neu': 0.684, 'pos': 0.247, 'co...",0.9578,1
4,Andy's new veterans jobs plan,"July 4, 2019","John,Tomorrow is the Fourth of July, a day for...",Democratic,United States,Kentucky,State Governor,12:15 AM,AM,12,0,July,4,2019,7,john tomorrow fourth july day us come together...,"{'neg': 0.023, 'neu': 0.636, 'pos': 0.341, 'co...",0.9867,1
5,say NO to Trump's military parade [signature n...,"July 4, 2019","Tomorrow is the Fourth of July, and Donald Tru...",Democratic,United States,NaN,President of the United States,12:19 AM,AM,12,0,July,4,2019,7,tomorrow fourth july donald trump using day th...,"{'neg': 0.078, 'neu': 0.684, 'pos': 0.238, 'co...",0.8316,1


In [4]:
df = df[df['year']>1970]
df['locality'] = np.where(df['locality']=="MIchigan","Michigan",df['locality'])
df.shape

(73694, 19)

In [5]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(["uh","oh","okay","im","dont"])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
df['body'] = df['body'].astype('string')


In [7]:
def remove_brackets_contents(text):
    pattern = r"\[.*?\]"
    return re.sub(pattern, " ", text)


def remove_special_characters(text):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return re.sub(pat, ' ', text)

In [8]:
import warnings
warnings.filterwarnings('ignore')

df['body'] = df['body'].replace('\n','',regex=True)
df['body'] = df['body'].replace('\r','',regex=True)
df.loc[:,'body'] = df['body'].apply(func = remove_brackets_contents)
df.loc[:,'body'] = df['body'].apply(func = remove_special_characters)


df['body'] = df['body'].replace('   ',' ',regex=True)
df['body'] = df['body'].replace('    ',' ',regex=True)
df['body'] = df['body'].replace('     ',' ',regex=True)

df['body'] = df['body'].replace(']',' ',regex=True)
df['body'] = df['body'].replace('\xa0',' ',regex=True)
df['body'] = df['body'].replace('almostdaily','almost daily',regex=True)


#Remove all the text that follows these sentences
df['body'] = df['body'].str.replace("(If you need to remove yourself from our email).*","")
df['body'] = df['body'].str.replace("(Email is the most important way).*","")
df['body'] = df['body'].str.replace("(This email was sent to).*","")
df['body'] = df['body'].str.replace("(Thank you for speaking).*","")
df['body'] = df['body'].str.replace("(All Rights Reserved).*","")
df['body'] = df['body'].str.replace("(mailto:).*","")
df['body'] = df['body'].str.replace("(Thank you for choosing to fight).*","")
df['body'] = df['body'].str.replace("(Paid for by).*","")
df['body'] = df['body'].str.replace("(We look forward to hearing from you).*","")
df['body'] = df['body'].str.replace("(PAID FOR BY).*","")
df['body'] = df['body'].str.replace("(Thanks for your).*","")
df['body'] = df['body'].str.replace("(All rights reserved).*","")
df['body'] = df['body'].str.replace("(unsubscribe).*","")
df['body'] = df['body'].str.replace("(Sincerely).*","")
df['body'] = df['body'].str.replace("(Facebook).*","")
df['body'] = df['body'].str.replace("(Twitter).*","")
df['body'] = df['body'].str.replace("(Please, sign your name).*","")
df['body'] = df['body'].str.replace("(CONTRIBUTE).*","")
df['body'] = df['body'].str.replace("(Contribute).*","")


In [10]:

from nltk.tokenize import RegexpTokenizer
!pip install contractions
import contractions

def punct(text):
    token=RegexpTokenizer(r'\w+')#regex
    text = token.tokenize(text)
    text= " ".join(text)
    return text 

def remove_special_characters(text):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return re.sub(pat, '', text)

def remove_digits(text):
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]' 
    return re.sub(pattern, '', text)

def lemmatize(text):
    lemmed = contractions.fix(str(text))
    return lemmed



def remove_weird_words_that_showed_up_in_analysis_(str):
    #select english stopwords
    #cachedStopWords = set(stopwords.words("english"))
    #add custom words
    cachedStopWords = set(('height','width','style','saved','payment','information','https',
                            'fmailchi','mp','chip','tr','td','style','font','size','px','view'
                            'email','browser','zwnj','metered','paywall','articles','vertical',
                            'align','top','item','img','src','rss','mc','actblue','com','donate',
                            'donation', 'padding','bottom', 'members','pre','unsubscribe','rsquo',
                            'headline','lead story','body lead','view email','story body','copy paste',
                            'address window','web update','web page','receiving email','page go',
                            'go link','paste link','copy fact','receiving newsletter','anymore click',
                            'newsletter shown','newsletter clicking','interested anymore','emails view',
                            'html emails','software display','display html','please click','web updates',
                            'email','emails','interest','interested','newsletter'
                           ))
    #remove stop words
    new_str = ' '.join([word for word in str.split() if word not in cachedStopWords]) 
    return new_str

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 8.7 MB/s 
     |████████████████████████████████| 110 kB 57.9 MB/s 


In [11]:
#Apply standardizing functions to body of email column
from nltk.corpus import stopwords

stop = stopwords.words('english')

#0.) Convert everything to string
df['cleaned_body'] = df['body'].astype(str)
#1.) Lowercase
df.loc[:,'cleaned_body'] = df['cleaned_body'].str.lower()
#2.) Remove punctuation
df.loc[:,'cleaned_body'] = df['cleaned_body'].apply(func = punct)
#3.) Remove non alpha-numeric characters
df.loc[:,'cleaned_body'] = df['cleaned_body'].apply(func = remove_special_characters)
#4.) Remove digits
df.loc[:,'cleaned_body'] = df['cleaned_body'].apply(func = remove_digits)
#5.) Lemmatize words
df.loc[:,'cleaned_body'] = df['cleaned_body'].apply(func = lemmatize)
#6. Remove stop words
#df.loc[:,'cleaned_body'] = df['cleaned_body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#7. Remove weird words that showed up in the analysis
df.loc[:,'cleaned_body'] = df['cleaned_body'].apply(func = remove_weird_words_that_showed_up_in_analysis_)

#Present results
df['cleaned_body'].head(5)


1    dear friend we have all seen the images and re...
2    team access to education is fundamental to a c...
3    beto for america john the trump administration...
4    john tomorrow is the fourth of july a day for ...
5    tomorrow is the fourth of july and donald trum...
Name: cleaned_body, dtype: object

In [12]:
df_dom = df[df['country']=="United States"]

DEM_df = df_dom[df_dom['party']=="Democratic"].reset_index(drop=True)
GOP_df = df_dom[df_dom['party']=="Republican"].reset_index(drop=True)

print(DEM_df.shape)
print(GOP_df.shape)

(27737, 19)
(6538, 19)


In [13]:
DEM = DEM_df.cleaned_body
GOP = GOP_df.cleaned_body

In [15]:
def clean_data(doc):
    doc = "".join(doc)
    doc = re.sub(r'[^\w\s]', '', doc)
    tokens = doc.split()
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word.isalpha() ]
    #tokens = [word for word in tokens if not word in  stop_words]
    text = " ".join(tokens)
    return tokens#,text

In [16]:
def return_lines(tokens,length):
    lines = []
    for i in range(length,len(tokens)):
        seq = tokens[i-length:i]
        line  = " ".join(seq)
        lines.append(line)
        
    return lines

In [17]:
dem_tokens = clean_data(DEM)
gop_tokens = clean_data(GOP)

dem_lines = return_lines(dem_tokens,10)
gop_lines = return_lines(gop_tokens,10)

In [18]:
def prepare_data(data):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(data)
    sequences = tokenizer.texts_to_sequences(data)
    sequences = np.array(sequences)
    X, y = sequences[:,:-1], sequences[:,-1]
    vocab_size = len(tokenizer.word_index) + 1
    return X,y,vocab_size,tokenizer

In [19]:
dem_X,dem_y,dem_vocab_size,dem_tokenizer = prepare_data(dem_lines)
seq_dem = dem_X.shape[1] 

gop_X,gop_y,gop_vocab_size,gop_tokenizer = prepare_data(gop_lines)
seq_gop = gop_X.shape[1] 

In [20]:
def model_rnn(X,y,epochs,vocab_size,seq):
    model = keras.Sequential([keras.layers.Embedding(input_dim = vocab_size,output_dim=20,input_length=seq),
                          keras.layers.Bidirectional(keras.layers.LSTM(100)),
                          keras.layers.Dense(vocab_size,activation="softmax")])
    
    
    model.compile(loss="sparse_categorical_crossentropy",metrics="accuracy",optimizer="adam")
    early_stop = keras.callbacks.EarlyStopping(patience=5)
    
    model.fit(X,y,epochs=epochs,callbacks=early_stop)
    
    return model

In [21]:
model = model_rnn(gop_X,gop_y,20,gop_vocab_size,seq_gop)

Epoch 1/20
46492/46492 [==============================] - ETA: 0s - loss: 5.7268 - accuracy: 0.1689

46492/46492 [==============================] - 548s 12ms/step - loss: 5.7268 - accuracy: 0.1689
Epoch 2/20
46490/46492 [============================>.] - ETA: 0s - loss: 4.8489 - accuracy: 0.2581

46492/46492 [==============================] - 531s 11ms/step - loss: 4.8489 - accuracy: 0.2581
Epoch 3/20
46488/46492 [============================>.] - ETA: 0s - loss: 4.5591 - accuracy: 0.2842

46492/46492 [==============================] - 542s 12ms/step - loss: 4.5591 - accuracy: 0.2842
Epoch 4/20
46491/46492 [============================>.] - ETA: 0s - loss: 4.3932 - accuracy: 0.2979

46492/46492 [==============================] - 538s 12ms/step - loss: 4.3932 - accuracy: 0.2979
Epoch 5/20
46491/46492 [============================>.] - ETA: 0s - loss: 4.2774 - accuracy: 0.3069

...

46492/46492 [==============================] - 534s 11ms/step - loss: 3.7410 - accuracy: 0.3499
Epoch 17/20
46492/46492 [==============================] - ETA: 0s - loss: 3.7155 - accuracy: 0.3518

46492/46492 [==============================] - 538s 12ms/step - loss: 3.7155 - accuracy: 0.3518
Epoch 18/20
46489/46492 [============================>.] - ETA: 0s - loss: 3.6910 - accuracy: 0.3539

46492/46492 [==============================] - 533s 11ms/step - loss: 3.6910 - accuracy: 0.3539
Epoch 19/20
46490/46492 [============================>.] - ETA: 0s - loss: 3.6673 - accuracy: 0.3564

46492/46492 [==============================] - 533s 11ms/step - loss: 3.6673 - accuracy: 0.3564
Epoch 20/20
46490/46492 [============================>.] - ETA: 0s - loss: 3.6404 - accuracy: 0.3586

46492/46492 [==============================] - 533s 11ms/step - loss: 3.6403 - accuracy: 0.3586


In [23]:
seed_text = "democrats"
num_word = 50
def new_email(seed_text,num_word,model,seq):
    for _ in range(num_word):
        tokens = gop_tokenizer.texts_to_sequences([seed_text])[0]
        tokens = pad_sequences([tokens],maxlen=seq,padding="post")
    
        predicted = np.argmax(model.predict(tokens))
        outputword = ""
    
        for word,index in gop_tokenizer.word_index.items():
            if predicted == index:
                outputword = word
                break
            
        seed_text += " " + outputword
    return seed_text

new_email(seed_text,num_word,model,seq_gop)

1/1 [==============================] - 0s 16ms/step


'democrats killing killing killing killing raises schools opposing abortions and impose they have a sick of crime and the rest of the democratic party has been recognized by the right direction to the united states mexico canada agreement usmca is a total of the second amendment bans socialism and they want'